In [ ]:
# !pip install ckiptagger

In [ ]:
# from ckiptagger import data_utils
# data_utils.download_data_gdown("./")

In [ ]:
import pandas as pd
tbl_post = pd.read_csv('tbl_post.csv')

In [ ]:
for i in range(len(tbl_post)):
    temp = tbl_post['Content'][i].replace("['", '').replace("']",'')
    tbl_post['Content'][i] = temp
tbl_post

In [ ]:
# 切詞用
def strQ2B(ustring):
    """把字串全形轉半形"""
    ss = []
    for s in ustring:
        rstring = ""
        for uchar in s:
            inside_code = ord(uchar)
            if inside_code == 12288:  # 全形空格直接轉換
                inside_code = 32
            elif (inside_code >= 65281 and inside_code <= 65374):  # 全形字元（除空格）根據關係轉化
                inside_code -= 65248
            rstring += chr(inside_code)
        ss.append(rstring)
    return ''.join(ss)

# 停用詞
# with open('stopwords_tc.txt', encoding='utf-8', mode='r') as f:
#     stop_words = []
#     for l in f:
#         stop_words.append(l.strip())

### -----------------------工作區----------------------------

In [ ]:
# 切詞
from ckiptagger import WS, POS, NER
import pandas as pd
# max(tbl_post['product_id'].unique())==1067-product_id最大
max_product_id = max(tbl_post['product_id'].unique())

ws = WS("./data")
pos = POS("./data")
Content_cut = []
Positive_cut = []

for i in range(max_product_id+1):
    # 同產品->product
    product1 = tbl_post[tbl_post['product_id']==i]
    product = product1.reset_index(drop=True)
    per_word_cut = []
    per_pos_cut = []
    
    for j in range(len(product)):
        ws_results = ws([product['Content'][j]])
        pos_results = pos(ws_results)
        
        for word in ws_results[0]:
            word = strQ2B(word)
            per_word_cut.append(word)
            
        for pos_word in pos_results[0]:
            per_pos_cut.append(pos_word)
                    
    Content_cut.append([per_word_cut])
    print(Content_cut)
    Positive_cut.append(per_pos_cut)
    print(Positive_cut)
    
Content_cutting = pd.DataFrame(Content_cut)
Positive_cutting = pd.DataFrame(Positive_cut)
Content_cutting['part of speech'] = Positive_cut
Content_cutting

In [ ]:
# 處理
product_id_Index=[]
max_product_id = max(tbl_post['product_id'].unique())
for i in range(max_product_id):
    i = i+1
    product_id_Index.append(i)
product_id_Index = pd.DataFrame(product_id_Index)
product_id_Index['content_cut'] = Content_cutting[0]
product_id_Index['part of speech'] = Content_cutting['part of speech']
product_id_Index.rename(columns = {0:'product_id'}, inplace = True)
product_id_Index
# 將該id沒有爬蟲內容者過濾
product_id_Index_final = product_id_Index[product_id_Index.content_cut.str.len()>0].reset_index(drop=True)
product_id_Index_final
product_id_Index_final.to_csv('Content_cut.csv', index=False, encoding="utf-8")